# 06 Ethics Analysis (Fairness)

**Senior Data Scientist:** En este notebook, realizaremos un análisis ético y de sesgos de nuestro modelo final de predicción de enfermedades cardíacas. 
El objetivo es evaluar si el modelo (`models/best_pipeline.pkl`) presenta disparidades significativas en su rendimiento a través de diferentes subgrupos demográficos, específicamente centrados en **Sexo** y **Edad**.

Evaluaremos métricas críticas para el dominio médico:
- **Falsos Negativos (FN):** Pacientes enfermos clasificados erróneamente como sanos. (Alto riesgo de salud).
- **Falsos Positivos (FP):** Pacientes sanos clasificados erróneamente como enfermos. (Ansiedad innecesaria y costos).

Este análisis corresponde al **Issue 40: Análisis Ético y de Sesgos**.

## 1. Configuración y Carga de Datos

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from fairlearn.metrics import MetricFrame, selection_rate, false_negative_rate, false_positive_rate
import sys
import os

# Aseguramos path para imports si fuera necesario
if os.path.abspath("..") not in sys.path:
    sys.path.append(os.path.abspath(".."))

pd.set_option('display.max_columns', None)

# Definir función auxiliar para cargar modelo
def load_model(path):
    return joblib.load(path)

# Cargar datos
data_path = "../data/02_intermediate/process_data.parquet"
df = pd.read_parquet(data_path)

# Cargar modelo final
# Intentamos cargar best_pipeline.pkl, fallback a final_pipeline_v1.pkl
model_path = "../models/best_pipeline.pkl"
if not os.path.exists(model_path):
    model_path = "../models/final_pipeline_v1.pkl"

print(f"Cargando modelo desde: {model_path}")
pipeline = load_model(model_path)

# Identificar target y features
target_col = 'HeartDisease' if 'HeartDisease' in df.columns else 'TARGET'
y_true = df[target_col]
X = df.drop(columns=[target_col])

## 2. Preparación de Grupos Sensibles

Identificamos las variables sensibles. Según la documentación:
- **Sexo (Sex):** 0 = Mujer, 1 = Hombre (o viceversa, verificaremos metadata).
- **Edad (Age):** Convertiremos la edad continua en grupos (bins) para análisis.

In [ ]:
# Verificar nombres de columnas en español/inglés
print("Columnas disponibles:", df.columns.tolist())

# Mapeo si es necesario
sex_col = 'Sex' if 'Sex' in df.columns else 'Sexo'
age_col = 'Age' if 'Age' in df.columns else 'Edad'

# Crear grupos de edad
df['Age_Group'] = pd.cut(df[age_col], bins=[0, 40, 60, 80, 100], labels=['<40', '40-60', '60-80', '80+'])

# Diccionario para etiquetas de sexo (asumiendo 1=Male, 0=Female según memory)
sex_labels = {0: 'Female', 1: 'Male'}
df['Sex_Label'] = df[sex_col].map(sex_labels)

print(df[['Sex_Label', 'Age_Group']].value_counts())

## 3. Generación de Predicciones

Obtenemos las predicciones del modelo sobre todo el dataset para evaluar el rendimiento global vs. subgrupos.

In [ ]:
y_pred = pipeline.predict(X)
y_pred_proba = pipeline.predict_proba(X)[:, 1] if hasattr(pipeline, "predict_proba") else y_pred

## 4. Análisis con Fairlearn

Utilizamos `MetricFrame` de Fairlearn para comparar métricas entre grupos.

### 4.1 Análisis por Sexo

In [ ]:
metrics = {
    'accuracy': accuracy_score,
    'recall (TPR)': recall_score,
    'false_negative_rate': false_negative_rate,
    'false_positive_rate': false_positive_rate
}

mf_sex = MetricFrame(
    metrics=metrics,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=df['Sex_Label']
)

print("Métricas por Sexo:")
print(mf_sex.by_group)

mf_sex.by_group.plot(kind='bar', subplots=True, layout=(2, 2), figsize=(10, 8), title="Métricas de Equidad por Sexo")
plt.show()

### 4.2 Análisis por Grupo de Edad

In [ ]:
mf_age = MetricFrame(
    metrics=metrics,
    y_true=y_true,
    y_pred=y_pred,
    sensitive_features=df['Age_Group']
)

print("Métricas por Edad:")
print(mf_age.by_group)

mf_age.by_group.plot(kind='bar', subplots=True, layout=(2, 2), figsize=(12, 8), title="Métricas de Equidad por Edad")
plt.show()

## 5. Análisis de Disparidad

Calculamos la diferencia máxima entre grupos para identificar el sesgo más pronunciado.

In [ ]:
print("Diferencia Máxima por Sexo:")
print(mf_sex.difference())

print("\nDiferencia Máxima por Edad:")
print(mf_age.difference())

## 6. Conclusiones y Recomendaciones

**Resumen de Hallazgos:**
*(Este espacio se debe completar con la interpretación de los resultados anteriores. Ejemplo:)*
- Si la Tasa de Falsos Negativos es significativamente mayor en un grupo (ej. Mujeres o Adultos Mayores), el modelo está fallando en detectar la enfermedad en ese grupo, lo cual es crítico.
- Si la Tasa de Falsos Positivos es alta en un grupo, estamos generando alertas innecesarias.

**Impacto Ético:**
- Un FNR desbalanceado podría llevar a una inequidad en el acceso a tratamiento preventivo.

**Siguientes Pasos:**
- Considerar re-entrenar con técnicas de mitigación de sesgo (ej. re-weighting) si las diferencias superan el 10-20%.